From: Maria Jensen, Agronomy Lead — NorthStar Ag Retail
To: You (Data Science Consultant)
Subject: Did our fungicide & insecticide applications actually pay off?

Hi there,
We ran foliar fungicide and insecticide passes across portions of our growers’ corn fields this season, based on scouting and weather conditions around VT/R1. The team wants to know whether these applications truly increased yield, and under which conditions they were most effective.

Specifically, could you:

Estimate the causal effect (bu/acre and $/acre) of fungicide, insecticide, and both, versus no spray, while controlling for confounders like disease/pest pressure, humidity, recent rain, crop stage, soil (OM/CEC/pH), nitrogen rate, planting density, and prior yield history.

Identify heterogeneous treatment effects (HTE): when do these treatments pay off? e.g., high humidity + high disease pressure near R1, certain hybrids, certain soils, etc.

Provide prescriptive guidance for next season: a simple decision rule or dashboard indicating when we should spray (and when we should skip) to maximize profit per acre, net of product and application cost.

We’re not looking for simple correlations; please use an approach like propensity scores with doubly robust estimation, or Causal Forests, to correct for non-random treatment decisions. A clear ROI analysis ($/acre) by condition is essential for our grower meetings next month.

Thanks!
— Maria

# Step 1 — Imports & Dataset Load

In [2]:
# Core Libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats / Modeling
from scipy import stats
import statsmodels.api as sm

# Display Settings
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 150)
sns.set(style="whitegrid", palette="viridis")

# Load Dataset
df = pd.read_csv(r'C:\Users\wallj\DS_Projects\Datasets\pest_treatment_attribution.csv')

# Quick shape & preview
print(f"Rows: {df.shape[0]:,}, Columns: {df.shape[1]}")
df.head()


Rows: 500, Columns: 31


,season_year,field_id,subfield_id,acres,hybrid,hybrid_susceptibility,crop_stage_window,disease_pressure_index,pest_pressure_index,scouting_score_0_100,gdd_to_date,precip_14d_in,humidity_7d_pct,soil_om_pct,soil_cec_meq,soil_ph,slope_pct,elevation_m,prior_yield_3yr_avg_bu_ac,planting_date,planting_density_per_ac,nitrogen_rate_lb_ac,treated_fungicide,treated_insecticide,treatment_timing_days_from_R1,treatment_cost_per_ac,yield_bu_ac,grain_moisture_pct,harvest_date,price_per_bu,observed_revenue_per_ac
0,2024,F01,F01_S25,7.8,Brevant-B07G,0.500,VT,0.234,0.000,10,1225,3.82,69,2.6,23.1,6.68,1.37,336.6,196.1,2024-05-01,32107,194,0,0,NaN,0.00,184.03,19.8,2024-10-15,4.75,874.13
1,2024,F01,F01_S24,7.9,Croplan-3899,0.382,VT,0.000,0.000,16,1126,3.44,55,2.6,23.1,6.68,1.37,336.6,196.1,2024-05-01,32107,194,1,0,0.0,15.96,196.11,20.0,2024-10-08,4.75,915.55
2,2024,F01,F01_S23,8.4,Croplan-3899,0.382,V14,0.221,0.151,32,1546,3.54,71,2.6,23.1,6.68,1.37,336.6,196.1,2024-05-01,32107,194,0,0,NaN,0.00,182.59,19.1,2024-10-17,4.75,867.33
3,2024,F01,F01_S22,11.3,HX-1132,0.430,R1,0.000,0.000,16,1218,1.37,63,2.6,23.1,6.68,1.37,336.6,196.1,2024-05-01,32107,194,0,0,NaN,0.00,197.78,16.3,2024-10-19,4.75,939.48
4,2024,F01,F01_S21,9.3,Brevant-B07G,0.500,R1,0.000,0.163,17,1432,3.51,81,2.6,23.1,6.68,1.37,336.6,196.1,2024-05-01,32107,194,0,0,NaN,0.00,183.21,20.9,2024-10-18,4.75,870.26


# Step 2 — Basic Info and Summary Stats

In [3]:
# Quick info
df.info()

# Numeric summary
df.describe().T.round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   season_year                    500 non-null    int64  
 1   field_id                       500 non-null    object 
 2   subfield_id                    500 non-null    object 
 3   acres                          500 non-null    float64
 4   hybrid                         500 non-null    object 
 5   hybrid_susceptibility          500 non-null    float64
 6   crop_stage_window              500 non-null    object 
 7   disease_pressure_index         500 non-null    float64
 8   pest_pressure_index            500 non-null    float64
 9   scouting_score_0_100           500 non-null    int64  
 10  gdd_to_date                    500 non-null    int64  
 11  precip_14d_in                  500 non-null    float64
 12  humidity_7d_pct                500 non-null    int

,count,mean,std,min,25%,50%,75%,max
season_year,500.0,2024.00,0.00,2024.00,2024.00,2024.00,2024.00,2024.00
acres,500.0,8.12,2.04,3.00,6.70,8.20,9.40,13.80
hybrid_susceptibility,500.0,0.53,0.12,0.38,0.43,0.50,0.56,0.75
disease_pressure_index,500.0,0.18,0.13,0.00,0.07,0.16,0.27,0.60
pest_pressure_index,500.0,0.18,0.12,0.00,0.10,0.18,0.26,0.51
scouting_score_0_100,500.0,22.66,11.97,0.00,15.00,22.00,31.00,61.00
gdd_to_date,500.0,1312.22,119.87,1000.00,1223.00,1313.00,1390.50,1600.00
precip_14d_in,500.0,2.83,1.14,0.00,2.07,2.85,3.65,6.03
humidity_7d_pct,500.0,71.50,7.79,45.00,66.00,71.00,77.00,94.00
soil_om_pct,500.0,3.93,0.83,2.22,3.58,4.16,4.48,5.10


# Missing Values Check

In [4]:
# Percentage of missing values per column
missing = df.isnull().mean().sort_values(ascending=False)
missing[missing > 0]

treatment_timing_days_from_R1    0.526
dtype: float64

# Treatment Overview

In [5]:
# Treatment flags
treat_cols = ["treated_fungicide", "treated_insecticide"]
treat_summary = df[treat_cols].apply(lambda x: x.value_counts(normalize=True)).T * 100
treat_summary.columns = ["% No", "% Yes"]
treat_summary.round(1)

,% No,% Yes
treated_fungicide,62.6,37.4
treated_insecticide,76.8,23.2


In [6]:
df["treatment_combo"] = (
    df["treated_fungicide"].astype(str) + df["treated_insecticide"].astype(str)
)
combo_map = {"00": "None", "10": "Fungicide Only", "01": "Insecticide Only", "11": "Both"}
df["treatment_combo"] = df["treatment_combo"].map(combo_map)

df["treatment_combo"].value_counts(normalize=True).mul(100).round(1)

treatment_combo
None                52.6
Fungicide Only      24.2
Both                13.2
Insecticide Only    10.0
Name: proportion, dtype: float64